### About
### Generate response for the question from pre-defined text using LLM(Extractive Question-Answering(QA) Model).

### I have directly used a pre-trained model without fine-tuning it on the training data, primarily due to my limited knowledge in NLP-QA tasks. I loaded and predicted the test data using the transformers inference pipeline.

### Install Necessary Libraries & Packages

In [ ]:
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm_notebook
import seaborn as sns
from datasets import Dataset
import re
import shutil
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


### Load test data

In [2]:
test=pd.read_csv("/kaggle/input/mh-intel-oneapi-hackathon-the-llm-challenge/test.csv")

### Load deepset_roberta-base-squad2 model

In [9]:
model_name = 'deepset/roberta-base-squad2'

### Transformers Inference pipeline

In [10]:

nlp_qa = pipeline('question-answering', model=model_name, tokenizer=model_name,device=0)
QA_input = {
    'question': test['Question'][1888],
    'context': test['Story'][1888]
}
res = nlp_qa(QA_input)

### Sample content

In [11]:
test['Story'][1888]

'Bloomberg L.P. is a privately held financial software, data, and media company headquartered in Midtown Manhattan, New York City. Bloomberg L.P. was founded by Michael Bloomberg in 1981 with the help of Thomas Secunda, Duncan MacMillan, Charles Zegar, and a 30% ownership investment by Merrill Lynch. \n\nBloomberg L.P. provides financial software tools such as an analytics and equity trading platform, data services, and news to financial companies and organizations through the Bloomberg Terminal (via its Bloomberg Professional Service), its core revenue-generating product. Bloomberg L.P. also includes a wire service (Bloomberg News), a global television network (Bloomberg Television), digital websites, a radio station (WBBR), subscription-only newsletters, and three magazines: "Bloomberg Businessweek", "Bloomberg Markets", and "Bloomberg Pursuits". In 2014, Bloomberg L.P. launched Bloomberg Politics, a multiplatform media property that merged the company\'s political news teams, and ha

### Sample question

In [12]:
test['Question'][1888]

'What does IMS stand for?'

### Sample answer generated from pipeline.

In [13]:
res

{'score': 0.9113449454307556,
 'start': 1346,
 'end': 1371,
 'answer': 'Innovative Market Systems'}

### Predict test data

In [ ]:
test_ans=[]
for i  in tqdm_notebook(range(test.shape[0])):
    QA_input = {
    'question': test['Question'][i],
    'context': test['Story'][i]
        }
    res = nlp_qa(QA_input)
    if res:
        test_ans.append(res['answer'])
    else:
        test_ans.append("unknown")

### Test Result

In [ ]:
res_final=pd.DataFrame({'Answer':test_ans})


In [ ]:
res_final.head(10)

In [ ]:
res_final.to_csv("deepset_roberta-base-squad2.csv",index=False)